In [10]:
from dotenv import load_dotenv, find_dotenv
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI


In [11]:
_ = load_dotenv(find_dotenv())

In [12]:
# using langchain message types
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, timeout=10)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")



## Passing a Pydantic class to LLM model

In [15]:
# Structured LLM
structured_llm = model.with_structured_output(Classification)

In [16]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

{'sentiment': 'contento', 'aggressiveness': 0, 'language': 'es'}

## Finer Control
Specifically, we can define:

Possible values for each property \
Description to make sure that the model understands the property \
Required properties to be returned \
Let's redeclare our Pydantic model to control for each of the previously mentioned aspects using enums

In [ ]:
from enum import Enum
class my_sentiment(str, Enum):
    HAPPY = "happy"
    NEUTRAL = "neutral"
    SAD = "sad"


class Classification1(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [34]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

# add structured output directly into the llm invokation
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification1
)

In [35]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

{'sentiment': 'contento', 'aggressiveness': 0, 'language': 'es'}

In [25]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

{'sentiment': 'enojado', 'aggressiveness': 8, 'language': 'es'}

In [36]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

{'sentiment': 'ok', 'aggressiveness': 0, 'language': 'English'}